In [1]:
cd /home/tkim/Academics/DistMEC/

/home/tkim/Academics/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm, tqdm_notebook
from time import sleep
import pickle
import os
import numpy as np
import itertools
import random

from sklearn.neighbors import KernelDensity
from scipy.stats import norm


from classes.distributed_utils import *


In [3]:
# Server Class
class Server():
    def __init__(self, rate, s_idx, location):
        self.rate = rate
        self.s_idx = s_idx
        self.location = location
        
        self.load_history = []
        self.num_offloader_history = []
        
    def receive_users(self, loads_list):
        load = np.sum(loads_list)
        rate = np.random.exponential(self.rate)
        latency = load/rate
        
        self.load_history += [load]
        self.num_offloader_history += [len(loads_list)]
        return latency
        
# User Class - Receive Reward + Arm Selection
class User():

    def __init__(self, load, latency_threshold, servers, T, latency_conversion, locs, 
                 max_dist = 7, threshold_dist = 6, self_weight = 0.5):
        
        num_servers = len(servers)
        
        self.load = load
        self.latency_threshold = latency_threshold
        self.pulls, self.means, self.ucb_idx = np.zeros(num_servers), np.zeros(num_servers), np.zeros(num_servers)
        self.t = int(0)
        self.reward_log = np.zeros(T)
        self.arm_history = np.zeros(T)
        self.num_servers = num_servers
        self.latency_conversion = latency_conversion # converts distance to time 
        
        self.locs = locs
        self.dists = self.get_dists()
        
        self.svr_locs = self.get_svr_locs(servers)
        self.usr_place = np.random.randint(len(locs))
        
        self.P_loc = None
        self.gen_MC(max_dist, threshold_dist, self_weight)
        
        self.loc_dists = None
        self.loc_lats = None
        self.loc_thresh = None 
        self.get_loc2serv_dist()
        
    def gen_MC(self, max_dist = 7, threshold_dist = 6, self_weight = 0.5):
        # Creating Markov Transition Probability Matrix 
        
        P = np.zeros(self.dists.shape)
        locs = self.locs
        for i in range(len(locs)):
            cut_list = self.dists[i,:]
            others = np.squeeze(np.argwhere((cut_list > 0) * (cut_list < threshold_dist) == True))
            num_others = others.shape[0]
        
            # Draw values to make up row of MC
            self_transition = np.random.exponential(scale=1/self_weight)
            others_transition = np.random.exponential(scale=1/((1-self_weight)*num_others),size=num_others)
            total = self_transition + np.sum(others_transition)
            
            P[i,i] = self_transition/total
            
            idx = 0
            for j in others:
                P[i,j] = others_transition[idx]/total
                idx += 1
            
        self.P_loc = P
        self.next_loc()
        
    def get_dists(self):
        # Obtaining distance matrix (from loc to loc) 
        
        locs = self.locs
        
        num_locs = len(locs)
        dists = np.zeros([num_locs,num_locs])
        
        for i,j in itertools.product(range(num_locs), range(num_locs)):
            if dists[i,j] == 0 and i != j:
                a = np.array(locs[i])
                b = np.array(locs[j])
                dists[i,j] = np.linalg.norm(a-b)
                dists[j,i] = dists[i,j]
        
        return dists
    
    def next_loc(self):
        # Update user location based on markov chain
        weights = self.P_loc[self.usr_place]
        population = range(weights.shape[0])
        self.usr_place =  random.choices(population, weights)[0]
    
    def get_svr_locs(self, servers):
        server_locs = []
        
        for s in range(len(servers)):
            server_locs += [servers[s].location]
            
        return server_locs
    
    def get_loc2serv_dist(self):
        
        # Getting distance from each user location to each server
        temp_loc_dists = []
        
        locs = self.locs
        svr_locs = self.svr_locs
        
        for u_loc in locs:
            temp_u_loc = []
            
            for s_loc in svr_locs:
                a = np.array(u_loc)
                b = np.array(s_loc)
                temp_u_loc += [np.linalg.norm(a-b)]
            
            temp_loc_dists += [temp_u_loc]
            
        self.loc_dists = temp_loc_dists
        
        # changing distance to latency
        temp_loc_lats = []
        
        for u in range(len(locs)):
            temp_u_lat = []
            
            for s in range(len(svr_locs)):
                temp_u_lat += [temp_loc_dists[u][s] * self.latency_conversion]
                
            temp_loc_lats += [temp_u_lat]
            
        self.loc_lats = temp_loc_lats
        
        # Changing latency to threshold
        temp_loc_thresh = []
        
        for u in range(len(locs)):
            temp_u_thresh = []
            
            for s in range(len(svr_locs)):
                temp_u_thresh += [max(self.latency_threshold - temp_loc_lats[u][s],0)]
            
            temp_loc_thresh += [temp_u_thresh]
         
        self.loc_thresh = temp_loc_thresh
            
    def select_arm_closest(self):
        # Baseline algorithm
        temp_max = np.array(self.loc_dists[self.usr_place]).min()
        arm_id = np.random.choice(np.flatnonzero(self.loc_dists[self.usr_place] == temp_max))
        self.arm_history[self.t] = int(arm_id)

    
        return arm_id
    
    def select_arm_random(self):
        arm_id = np.random.choice(range(self.num_servers))
        self.arm_history[self.t] = int(arm_id)
        
        return arm_id
    
    def select_arm_dist(self, server_dists, server_rates): # Assuming linear, dists takes Vs value
        
        prob_lists = np.zeros(self.num_servers)
        
        for i in range(self.num_servers):
            
            lat = self.loc_thresh[self.usr_place][i]
            
            if lat > 0:
                L = server_dists[i]
                C = server_rates[i]
                prob_lists[i] = lat * (np.exp(-self.load * C/lat) - np.exp(-(self.load + L) * C/lat))/ (C * L)
            else:
                prob_lists[i] = 0
            
        arm_id = np.random.choice(np.flatnonzero(prob_lists == prob_lists.max()))
        self.arm_history[self.t] = int(arm_id)
        return arm_id

    
    def log_reward(self, latency):
        
        curr_reward = 0
        arm_id = int(self.arm_history[self.t])
        
        lat_compare = self.loc_thresh[self.usr_place][arm_id]
        if latency < lat_compare:
            curr_reward = 1

        self.reward_log[self.t] = curr_reward

        self.pulls[arm_id] += 1
        self.t += int(1)
        
        return curr_reward
    
def init_offload_dict(num_svrs):
    offload_dict = {}
    for i in range(num_svrs):
        offload_dict[i] = []
    return offload_dict



In [4]:
T = 5000
num_users = 60
num_svrs = 30
num_locs = 36
space_1d_dim = 100

# Draw discrete locations for servers and users in space
usr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)
svr_locs = gen_eq_locs(space_1d=space_1d_dim, nums=num_locs,offset = 1.7)

# Setting latency dependent values
thresh_u = np.random.uniform(2,2,num_users)
load_u = np.random.uniform(1,2,num_users)
svr_rate = np.random.uniform(1,1, num_svrs)
latency_conversion = 0.1

# Create Servers
Servers = []
for s in range(num_svrs):
    Servers += [Server(svr_rate[s], s, svr_locs[s])]

# Create Users
Users = []
for u in range(num_users):
    Users += [User(load_u[u], thresh_u[u], Servers, T, latency_conversion, usr_locs,
                  max_dist = 30, threshold_dist = 40, self_weight = 0.5)]

rewards = np.zeros(T)

# Run Bandits
for t in range(T):
    
    # Users select arms
    offload_dict = init_offload_dict(num_svrs)
    load_dict = init_offload_dict(num_svrs)
    for u in range(num_users):
        Users[u].next_loc()
        arm = Users[u].select_arm_closest()
        load = Users[u].load
        
        offload_dict[arm] += [u]
        load_dict[arm] += [load]
    
        
    # Offload to servers and get latency
    lat_vals = np.zeros(num_users)
    for s in range(num_svrs):
        lat = Servers[s].receive_users(load_dict[s])
        for u in offload_dict[s]:
            reward = Users[u].log_reward(lat)    
    
            rewards[t] += reward

mean_val = []

for s in Servers:
    print(np.mean(s.load_history))
    mean_val += [np.mean(s.load_history)]

1.3506360049164985
2.1414388774465456
2.707705744478039
2.1788414174839232
1.9932003145645976
1.1549523811685947
1.854312651883378
3.1892949343479424
3.350517813908564
2.9747820371086613
2.6187197722398254
1.8424555751301546
2.3207276177318947
2.977440648066617
4.14626324537052
3.671679204815944
3.2625627503845487
2.017110193387896
2.581869664298461
2.953394450206431
3.9035987955391924
3.736387718148579
3.3962916231895433
2.0220265060733205
3.3753277840665716
4.504799194141931
5.657747462858579
4.829494503724091
4.333672132321875
2.8720691212797838


In [5]:
# System Parameters
T = 5000
num_users = 61
num_svrs = 30

# thresh_u = np.random.uniform(1,1,num_users)
# load_u = np.random.uniform(0.5,1,num_users)
# svr_rate = np.random.uniform(1,1, num_svrs)

thresh_u2 = np.append(thresh_u, np.random.uniform(1,1,1))
load_u2 = np.append(load_u, np.random.uniform(0.5,1,1))

In [6]:
# All users naive

# Create Servers
Servers = []
for s in range(num_svrs):
    Servers += [Server(svr_rate[s], s, svr_locs[s])]

# Create Users
Users = []
for u in range(num_users):
    Users += [User(load_u2[u], thresh_u2[u], Servers, T, latency_conversion, usr_locs,
                  max_dist = 30, threshold_dist = 40, self_weight = 0.5)]

rewards = np.zeros(T)

# Run Bandits
for t in range(T):
    
    # Users select arms
    offload_dict = init_offload_dict(num_svrs)
    load_dict = init_offload_dict(num_svrs)
    for u in range(num_users):
        Users[u].next_loc()
        arm = Users[u].select_arm_closest()
        load = Users[u].load
        
        offload_dict[arm] += [u]
        load_dict[arm] += [load]
    
        
    # Offload to servers and get latency
    lat_vals = np.zeros(num_users)
    for s in range(num_svrs):
        lat = Servers[s].receive_users(load_dict[s])
        for u in offload_dict[s]:
            reward = Users[u].log_reward(lat)    
    
            rewards[t] += reward

# for s in Servers:
#     print(np.mean(s.load_history))
        
print('one user:', np.mean(Users[-1].reward_log))

reward_all = []
for u in Users:
    reward_all += [np.mean(u.reward_log)]
print('all user:', np.mean(reward_all))

one user: 0.0828
all user: 0.14437049180327868


In [7]:
# Last user smart

server_dists = np.array(mean_val) * 2
server_rates = 1/ svr_rate

# Create Servers
Servers = []
for s in range(num_svrs):
    Servers += [Server(svr_rate[s], s, svr_locs[s])]

# Create Users
Users = []
for u in range(num_users):
    Users += [User(load_u2[u], thresh_u2[u], Servers, T, latency_conversion, usr_locs,
                  max_dist = 30, threshold_dist = 40, self_weight = 0.5)]

rewards = np.zeros(T)

# Run Bandits
for t in range(T):
    
    # Users select arms
    offload_dict = init_offload_dict(num_svrs)
    load_dict = init_offload_dict(num_svrs)
    for u in range(num_users):
        Users[u].next_loc()
        
        if u < num_users-1:
            arm = Users[u].select_arm_closest()
        else:
            arm = Users[u].select_arm_dist(server_dists, server_rates)
        load = Users[u].load
        
        offload_dict[arm] += [u]
        load_dict[arm] += [load]
    
        
    # Offload to servers and get latency
    lat_vals = np.zeros(num_users)
    for s in range(num_svrs):
        lat = Servers[s].receive_users(load_dict[s])
        for u in offload_dict[s]:
            reward = Users[u].log_reward(lat)    
    
            rewards[t] += reward

# for s in Servers:
#     print(np.mean(s.load_history))
        
print('one user:', np.mean(Users[-1].reward_log))

reward_all = []
for u in Users:
    reward_all += [np.mean(u.reward_log)]
print('all user:', np.mean(reward_all))

one user: 0.0946
all user: 0.1447639344262295


In [8]:
# All user smart

server_dists = np.array(mean_val) * 2
server_rates = 1/ svr_rate

# Create Servers
Servers = []
for s in range(num_svrs):
    Servers += [Server(svr_rate[s], s, svr_locs[s])]

# Create Users
Users = []
for u in range(num_users):
    Users += [User(load_u2[u], thresh_u2[u], Servers, T, latency_conversion, usr_locs,
                  max_dist = 30, threshold_dist = 40, self_weight = 0.5)]

rewards = np.zeros(T)

# Run Bandits
for t in range(T):
    
    # Users select arms
    offload_dict = init_offload_dict(num_svrs)
    load_dict = init_offload_dict(num_svrs)
    for u in range(num_users):
        Users[u].next_loc()
        
        arm = Users[u].select_arm_dist(server_dists, server_rates)
        load = Users[u].load
        
        offload_dict[arm] += [u]
        load_dict[arm] += [load]
    
        
    # Offload to servers and get latency
    lat_vals = np.zeros(num_users)
    for s in range(num_svrs):
        lat = Servers[s].receive_users(load_dict[s])
        for u in offload_dict[s]:
            reward = Users[u].log_reward(lat)    
    
            rewards[t] += reward

mean_val_smart = []
for s in Servers:
#     print(np.mean(s.load_history))
    mean_val_smart += [np.mean(s.load_history)]
        
print('one user:', np.mean(Users[-1].reward_log))

reward_all = []
for u in Users:
    reward_all += [np.mean(u.reward_log)]
print('all user:', np.mean(reward_all))

one user: 0.0972
all user: 0.14722295081967218
